In [148]:
import pandas as pd

# no missing data
full = pd.DataFrame({
    'swims_like_a_duck':  [0,0,1,1,0,0,1,1, 1, 1],
    'quacks_like_a_duck': [0,1,0,1,0,1,0,1, 0, 1],
    'duck':               [0,0,0,0,0,0,0,1, 0, 1]
})

In [149]:
# some data is missing - denoted by -1
with_missing = pd.DataFrame({
    'swims_like_a_duck':  [0,0,1,1,0,0,1,1,   1, -1],
    'quacks_like_a_duck': [0,1,0,1,0,1,0,1,  -1, -1],
    'duck':               [0,0,0,1,0,0,0,1,   0, 1]
})

We expect the bayesian network to predict  
**+1** for the missing 'swims' value  
and  
**0, +1** for the two missing 'quacks' value

defining the network: unobserved variables first

In [150]:
import pymc
# prior probabilities for swimming and quacking
swim_prior = pymc.Uniform('P(swims)', lower=0, upper=1, size=1)
quack_prior = pymc.Uniform('P(quacks)', lower=0, upper=1, size=1)

# probability of being a duck conditional on swimming and quacking
# (or not swimming and quacking etc.)
p_duck_swim_quack = pymc.Uniform('P(duck | swims & quacks)', lower=0, upper=1, size=1)
p_duck_not_swim_not_quack = pymc.Uniform('P(duck | not swims & not quacks)', lower=0, upper=1, size=1)
p_duck_not_swim_quack = pymc.Uniform('P(duck | not swims & quacks)', lower=0, upper=1, size=1)
p_duck_swim_not_quack = pymc.Uniform('P(duck | swims & not quacks)', lower=0, upper=1, size=1)

Now add the observed nodes. Need to use masked arrays where there are missing values

In [151]:
import numpy as np
swim_data = with_missing.swims_like_a_duck
masked_swim_data = np.ma.masked_array(swim_data, swim_data == -1, fill_value=0)

quack_data = with_missing.quacks_like_a_duck
masked_quack_data = np.ma.masked_array(quack_data, quack_data == -1, fill_value=0)

duck_data = with_missing.duck
masked_duck_data = np.ma.masked_array(duck_data, duck_data == -1, fill_value=0)

In [152]:
masked_quack_data

masked_array(data = [0 1 0 1 0 1 0 1 -- --],
             mask = [False False False False False False False False  True  True],
       fill_value = 0)

In [153]:
# number of animal observations
n = len(with_missing)

swims = pymc.Bernoulli('swims', p=swim_prior, observed=True, value=masked_swim_data, size=n)
quacks = pymc.Bernoulli('quacks', p=quack_prior, observed=True, value=masked_quack_data, size=n)

In [154]:
# auxiliary pymc variable - probability of duck
@pymc.deterministic
def duck_probability(
        swims=swims, 
        quacks=quacks, 
        p_duck_swim_quack=p_duck_swim_quack,
        p_duck_not_swim_quack=p_duck_not_swim_quack, 
        p_duck_swim_not_quack=p_duck_swim_not_quack, 
        p_duck_not_swim_not_quack=p_duck_not_swim_not_quack):

    d = []
    for s, q in zip(swims, quacks):
        if (s and q):
            d.append(p_duck_swim_quack)
        elif (s and (not q)):
            d.append(p_duck_swim_not_quack)
        elif ((not s) and q):
            d.append(p_duck_not_swim_quack)
        elif ((not s) and (not q)):
            d.append(p_duck_not_swim_not_quack)
        else:
            raise ValueError('this should never happen')
            
    return np.array(d).ravel()

# AND FINALLY
duck = pymc.Bernoulli('duck', p=duck_probability, observed=True, value=masked_duck_data, size=n)


In [155]:
# putting it all together
model = pymc.Model([swims, quacks, duck])

## getting the result: method 1 - MAP

initial values of all variables:

In [156]:
print swims.value
print quacks.value
print duck.value
print swim_prior.value
print quack_prior.value
print p_duck_not_swim_quack.value
print p_duck_swim_quack.value

[False False  True  True False False  True  True  True False]
[False  True False  True False  True False  True False False]
[False False False  True False False False  True False  True]
[ 0.32001799]
[ 0.47628556]
[ 0.7691245]
[ 0.61535582]


optimising values:

In [157]:
pymc.MAP(model).fit(method='fmin')

and now optimised values:

In [158]:
print swims.value
print quacks.value
print duck.value
print swim_prior.value
print quack_prior.value
print p_duck_not_swim_quack.value
print p_duck_swim_quack.value

[False False  True  True False False  True  True  True False]
[False  True False  True False  True False  True False  True]
[False False False  True False False False  True False  True]
[ 0.33401878]
[ 0.47628556]
[ 0.7691245]
[ 0.61535582]


in general parameter values seem to have moved in the right direction, and the values of the last unobserved 'swim' and 'quack' flipped from False to True. That is good. But The second t olast example actually flipped in the wrong direction

 ¯\_(ツ)_/¯

### getting the result: method 2 - MCMC
sample with MCMC and look for most common values

In [159]:
# this will generate (10000 - 8000) / 10 = 200 samples
sampler = pymc.MCMC(model)
sampler.sample(iter=10000, burn=8000, thin=10)

 [-----------------100%-----------------] 10000 of 10000 complete in 4.1 sec

where there was a single unobserved variable, we have 200 samples:

In [160]:
sampler.trace('P(swims)')[:].shape

(200, 1)

In [161]:
sampler.trace('P(duck | not swims & quacks)')[:].shape

(200, 1)

where there was an observed variable with 2 missing values pymc created 2 unobserved variables and we have 200 samples for each:

In [162]:
sampler.trace('quacks')[:].shape

(200, 2)

In [163]:
sampler.trace('swims')[:].shape

(200, 1)

duck didn't have any missing values, so there won't be any samples:

In [164]:
sampler.trace('duck')[:].shape

KeyError: 'duck'

let's get the the average prediction for paramters:

In [165]:
sampler.trace('P(duck | not swims & quacks)')[:].mean()

0.25907298682219188

In [166]:
sampler.trace('P(duck | swims & quacks)')[:].mean()

0.74353922377038228

makes sense. 

And finally prediction for the missing values:

In [167]:
sampler.trace('quacks')[:].mean(axis=0)

array([ 0.23,  0.69])

In [168]:
sampler.trace('swims')[:].mean(axis=0)

array([ 0.725])

These are all as expected!


And this is how all the above steps can expressed succinctly using the utilities I developed:

In [171]:
from dstk.pymc_utils import make_bernoulli, cartesian_bernoulli_child
from dstk.imputation import BayesNetImputer

class DuckImputer(BayesNetImputer):
    def construct_net(self, df):
        quacks = make_bernoulli('quacks_like_a_duck', value=df.quacks_like_a_duck)
        swims = make_bernoulli('swims_like_a_duck', value=df.swims_like_a_duck)
        duck = cartesian_bernoulli_child('duck', parents=[quacks, swims], value=df.duck)
        return pymc.Model([quacks, swims, duck])
    
DuckImputer(method='MCMC').fit_transform(with_missing)

 [-----------------100%-----------------] 500 of 500 complete in 0.2 sec

,duck,quacks_like_a_duck,swims_like_a_duck
0,0,0,0
1,0,1,0
2,0,0,1
3,1,1,1
4,0,0,0
5,0,1,0
6,0,0,1
7,1,1,1
8,0,0,1
9,1,1,1


In [172]:
DuckImputer(method='MAP').fit_transform(with_missing)

,duck,quacks_like_a_duck,swims_like_a_duck
0,0,0,0
1,0,1,0
2,0,0,1
3,1,1,1
4,0,0,0
5,0,1,0
6,0,0,1
7,1,1,1
8,0,1,1
9,1,1,1
